# Week 3

## Week 3 Lesson 1
## Logistic Regression
5 - 6 June 2017

### Regression vs Classification
* If the y variable is numeric then we have a regression problem - we are trying to predict a continuous number
* If the y variable is a category (for example trying to predict a type of flower) the we have a classification problem - we are trying to classify what group that y belongs to.
* We want to build a classifier that correctly identifies which class our target variable y belongs to given our input variable x. 
* Why not use the linear regression model? y=Xβ+ϵ
 * If we only have a binary response variable (0 or 1) it might make sense… BUT we can have our estimated value of y > 1 or y < 0 … which doesn’t make sense.
 * What of the case where we have more than one class? Linear regression cannot easily handle these cases.
 * We want a classification method that can handle these cases and give us results we can easily interpret. 

<img src="img/logit-function.png" width=400 align=right>
### p(1|x) = β0+β1
* This is a good starting point but we still have the problem of p(Y) being outside the 0,1 range.
* We need to model p(Y=1|X) using a function that gives outputs between 0 and 1. Basically we want something that looks like the following:
<img src="img/logistic-regression.png" width=300 align=right>



* We can see that it this function is linear in X
 * p | (p - 1) is called the ‘odds’ and can be any value from 0 to ∞
 * log ( p | (p - 1) ) is called the ‘log-odds’ or ‘logit’

### Accuracy Score
* This is simply the fraction of correct predictions from the model. So it is the number of correct predictions divided by the number of observations in our dataset.

<img src="img/confusion-matrix.png" align=right>
### Confusion Matrix
* A confusion matrix shows us what the predicted class was against what the actual class was. 
* The true class makes up the rows or the vertical axes and the predicted class makes up the columns or the horizontal axis.
* Any entries in the diagonal of the matrix are those that are correctly classified. 


<img src="img/roc-curve.png" align=right>
### Receiver Operating Characteristic

* The Receiver Operating Characteristic or ROC curve shows the performance of a binary classifier system as its discrimination threshold is varied. 
* It is created by plotting the fraction of true positives out of the positives (TPR = true positive rate) vs. the fraction of false positives out of the negatives (FPR = false positive rate), at various threshold settings.
* By computing the Area Under the Curve of the ROC curve we get a single number summary of accuracy.
* The closer that number is to 1 the more accurate our model is. 


In [1]:
import pandas as pd
#read the data
titanic = pd.read_csv('../data/titanic.csv', index_col='PassengerId')
#define the columns to read (x), and the target (y)
feature_cols = ['Pclass', 'Parch']
X = titanic[feature_cols]
y = titanic.Survived
#split the data
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
#fit the logreg model and print the coefficients
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
zip(feature_cols, logreg.coef_[0])
print(logreg.fit(X_train, y_train))

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [2]:
#make predictions on testing set and test accuracy
y_pred_class = logreg.predict(X_test)
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))
#print the confusion matrix
from sklearn import metrics
prds = logreg.predict(X)
print(metrics.confusion_matrix(y_test, y_pred_class))

0.668161434978
[[105  23]
 [ 51  44]]


In [ ]:
#generate the ROC curve
import matplotlib.pyplot as plt
# Generate the prediction values for each of the test observations using predict_proba() function rather than just predict
preds = logreg.predict_proba(X_test)[:,1]
# Store the false positive rate(fpr), true positive rate (tpr) in vectors for use in the graph
fpr, tpr, _ = metrics.roc_curve(y_test, preds)
# Store the Area Under the Curve (AUC) so we can annotate our graph with this metric
roc_auc = metrics.auc(fpr,tpr)
# Plot the ROC Curve
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()
# What's happening here is we are changing the cutoff value from 0 to 1.
# When we have a cutoff of zero this means that we have no positive predictions so both fpr and tpr are both 0
# Our aim when modelling is to maximise the area under the curve, the closer to one the better the model.

In [ ]:
#split data into binaries where not binary
titanic_with_dummies = pd.get_dummies(data=titanic, columns = ['Sex', 'Embarked', 'Pclass'], prefix = ['Sex', 'Embarked', 'Pclass'] )
titanic_with_dummies['Age'] = titanic_with_dummies[['Age',"Parch","Sex_male",'Pclass_1', 'Pclass_2']].groupby(["Parch","Sex_male",'Pclass_1', 'Pclass_2'])['Age'].transform(lambda x: x.fillna(x.mean()))
feature_cols = ['Parch', 'Sex_male', 'Sex_female', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Fare', 'Age']
X = titanic_with_dummies[feature_cols]
y = titanic_with_dummies.Survived
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
logreg.fit(X_train, y_train)
zip(feature_cols, logreg.coef_[0])
y_pred_class = logreg.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred_class))
preds = logreg.predict_proba(X_test)[:,1]
fpr, tpr, _ = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr,tpr)
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

### KNN Classification

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN_model = KNeighborsClassifier(5)
KNN_model.fit(X_train, y_train)
y_pred_class = KNN_model.predict(X_test)
# Print the new accuracy rate
print(metrics.accuracy_score(y_test, y_pred_class))

In [ ]:
from sklearn.naive_bayes import GaussianNB
NB_model = GaussianNB()
NB_model.fit(X_train, y_train)
y_pred_class = NB_model.predict(X_test)
# Print the new accuracy rate
print(metrics.accuracy_score(y_test, y_pred_class))

### Additional readings
* Logistic Regression applied to loan applications https://github.com/nborwankar/LearnDataScience
* Odds Ratio in Logistic Regression http://www.ats.ucla.edu/stat/mult_pkg/faq/general/odds_ratio.htm

---

## Week 3 Lesson 2
## Model Evaluation
6 - 8 June 2017

### What is model evaluation?
* Model evaluation is how we test whether our model works on new data.

### Why do we do model evaluation?
* A model bsed on training data has a set error - the training error - for how well the model fits training data. This is a good way to see how the model fits _that_ data, but not all data. 
* We want to test other data on the model for a better measure of accuracy.
* Without model evaluation, you run the risk of over-fitting the training data, which will have high accuracy for the training data but low accuracy for everything else.
* This is what we call **bias**: if your model has too much bias, it will be accurate for the training data, but will be too closely fit to the data and so won't have the same accuracy for new data.
* **Variance** is the opposite: if your model has too much variance, it won't be accurate for the training data, because it's not fit well enough.
<img src="img/bias-and-variance.png">

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

#### High Variance, Low Bias:

In [ ]:
df = pd.read_table('http://people.sc.fsu.edu/~jburkardt/datasets/regression/x01.txt', sep='\s+', skiprows=33, names=['id','brain','body'], index_col='id')
np.random.seed(12345)
df = df[df.body < 200]
df['sample'] = np.random.randint(1, 3, len(df))

sns.lmplot(x='body', y='brain', data=df, ci=None, hue='sample')
sns.plt.xlim(-10, 200)
sns.plt.ylim(-10, 250)

#### Low Variance, High Bias

In [ ]:
sns.lmplot(x='body', y='brain', data=df, ci=None, hue='sample', order=8)
sns.plt.xlim(-10, 200)
sns.plt.ylim(-10, 250)

#### Good balance

In [ ]:
sns.lmplot(x='body', y='brain', data=df, ci=None, hue='sample', order=2)
sns.plt.xlim(-10, 200)
sns.plt.ylim(-10, 250)

### How do we do model evaluation?
* We do model evaluation by separating out training data and test data from our dataset, and using the training data to build the model. We then test its accuracy on test data. This is the out-of-sample error (OOS).
* If we had a different training/test split, the OOS error would be different.
* One way of improving this will be to use multiple splits for test and training data. Train the model with different splits of data, and test with the rest from that split.
* This is called **cross-validation**.

In [ ]:
#We'll be using logistic regression with Titanic dataset:
import pandas as pd
titanic = pd.read_csv('../data/titanic.csv', index_col='PassengerId')
#Imputing values for Age:
from sklearn.preprocessing import Imputer
imp = Imputer(strategy='mean', axis=1)
titanic['Age'] = imp.fit_transform(titanic.Age.values.reshape(1,-1)).T
#Logreg on Age:
feature_cols = ['Pclass', 'Parch','Age']
X = titanic[feature_cols]
y = titanic.Survived
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e9)
logreg.fit(X_train, y_train)
y_pred_class = logreg.predict(X_test)

#### Cross-validation
* Steps for K-fold cross-validation:
 1. Randomly split the dataset into K equal partitions.
 2. Use partition 1 as test set & union of other partitions as training set.
 3. Calculate test set error.
 4. Repeat steps 2-3 using a different partition as the test set at each iteration.
 5. Take the average test set error as the estimate of OOS accuracy.
* Features of K-fold cross-validation:
 1. More accurate estimate of OOS prediction error.
 2. More efficient use of data than single train/test split.
   * Each record in our dataset is used for both training and testing.
 3. Presents tradeoff between efficiency an computational expense.
   * 10-fold CV is 10x more expensive than a single train/test split
 4. Can be used for parameter tuning and model selection.

In [ ]:
titanicwd = pd.get_dummies(data=titanic, columns = ['Sex', 'Embarked', 'Pclass'], prefix = ['Sex', 'Embarked', 'Pclass'] )
# Fill any null values of age by taking average by gender, class and parch
titanicwd['Age']=titanicwd[["Age","Parch","Sex_male","Pclass_1","Pclass_2"]].groupby(["Parch","Sex_male",'Pclass_1', 'Pclass_2'])['Age'].transform(lambda x: x.fillna(x.mean()))
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
feature_cols = ['Sex_male', 'Sex_female','Embarked_C','Embarked_Q','Embarked_S','Pclass_1', 'Pclass_2', 'Pclass_3','Parch', 'Age','SibSp']
X = titanicwd[feature_cols]
y = titanicwd.Survived

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
logreg.fit(X_train, y_train)
zip(feature_cols, logreg.coef_[0])
y_pred_class = logreg.predict(X_test)

print(metrics.accuracy_score(y_test, y_pred_class))

In [ ]:
scores = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10)
print(scores)

In [ ]:
print(scores.mean())

In [ ]:
from sklearn import cross_validation
from sklearn import linear_model
mse_values = []
scores = []
n = 0

feature_cols = ['Sex_male', 'Sex_female','Embarked_C','Embarked_Q','Embarked_S','Pclass_1', 'Pclass_2', 'Pclass_3',]
modeldata = titanicwd[feature_cols]
y = titanicwd.Survived 

kf = cross_validation.KFold(len(modeldata), n_folds=10, shuffle=True)

print("~~~~ CROSS VALIDATION each fold ~~~~")
for train_index, test_index in kf:
    lm = linear_model.LinearRegression().fit(modeldata.iloc[train_index], y.iloc[train_index])
    mse_values.append(metrics.mean_squared_error(y.iloc[test_index], lm.predict(modeldata.iloc[test_index])))
    scores.append(lm.score(modeldata, y))
    n+=1
    print ('Model', n)
    print ('MSE:', mse_values[n-1])
    print ('R2:', scores[n-1])


print ("~~~~ SUMMARY OF CROSS VALIDATION ~~~~")
print ('Mean of MSE for all folds:', np.mean(mse_values))
print ('Mean of R2 for all folds:', np.mean(scores))

#### Null accuracy
* **Null accuracy** is the accuracy that could be achieved by always predicting the **most frequent class**. It is a baseline against which you may want to measure your classifier.

In [ ]:
print (y_test.mean())
print (1 - y_test.mean())

In [ ]:
from sklearn.dummy import DummyClassifier
dumb = DummyClassifier(strategy='most_frequent')
dumb.fit(X_train, y_train)
y_dumb_class = dumb.predict(X_test)
print (metrics.accuracy_score(y_test, y_dumb_class))

#### Confusion matrix
* Confusion Matrix is another way to evaluate a model. It is a table to describe the performance of a classifier.
* **Sensitivity | Recall** = (number of true positives) | (true positives + false negatives - as in, number of all real positives)
* **Specificity** = (number of true negatives) | (true negatives + false positives - as in number of all real negatives)

In [ ]:
# confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

In [ ]:
#load confusion_matrix_nice.py
#from confusion_matrix_nice import plot_confusion_matrix
%run confusion_matrix_nice
cnf_mat = metrics.confusion_matrix(y_test, y_pred_class, labels = titanic.Survived.unique())
class_labels = titanic.Survived.unique()
plt.figure()
plot_confusion_matrix(cnf_mat, class_labels,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues)
plt.show()

In [ ]:
#sensitivity = TP / TP + FN
#specificity = TN / TN + FP
sensitivity=43/(43+52)
specificity = 107/(107+21)
print(sensitivity,specificity)

In [ ]:
y_pred_prob = logreg.predict_proba(X_test)[:, 1]
%matplotlib inline
import matplotlib.pyplot as plt
plt.hist(y_pred_prob)
plt.xlabel('Predicted probability of survival')
plt.ylabel('Frequency')

In [ ]:
import numpy as np
y_pred_class = np.where(y_pred_prob > 0.25, 1, 0)

In [ ]:
print (metrics.confusion_matrix(y_test, y_pred_class))

In [ ]:
sensitivity = 68 / (27 + 68)
specificity = 57/ (57 + 71)
print(sensitivity,specificity)

#### Precision + recall
* **Precision** = (number of true positives) | (true positives + false positives - as in number of all predicted positives)
* **Sensitivity | Recall** = (number of true positives) | (true positives + false negatives - as in, number of all real positives)

In [ ]:
#Accuracy score returns the percent of true positives and true negatives samples (in decimal format with 1 being perfect)
print('Accuracy score:',metrics.accuracy_score(y_test, y_pred_class))
#Precision score returns the correctly identified 
# = # true positives / true positives + false positives:
print('Precision score:',metrics.precision_score(y_test, y_pred_class))
#Recall score returns the # of  positives correctly predicted 
# = # true positives / true positives + false negatives:
print('Recall score:',metrics.recall_score(y_test,y_pred_class))
# The F1 score can be interpreted as a weighted average of the precision and recall.
# An F1 score reaches its best value at 1 and worst score at 0. 
# The relative contribution of precision and recall to the F1 score are equal. 
# F1 = 2 * (precision * recall) / (precision + recall)
print('F1 score:',metrics.f1_score(y_test,y_pred_class))

### Additional readings
* Caltech's Learning From Data course visualising bias and variance (15 mins) http://work.caltech.edu/library/081.html
* Rahul Patwari has a great video on ROC Curves (12 minutes)  https://www.youtube.com/watch?v=21Igj5Pr6u4
* Have a look at scikit-learn’s documentation on model evaluation http://scikit-learn.org/stable/modules/model_evaluation.html

---

---